In [3]:
# install 없이 import 가능한 모듈

import numpy as np
import pandas as pd
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from transformers import pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import re
import nltk
from tqdm import tqdm, tqdm_notebook
import bz2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [18]:
?ModelCheckpoint

In [5]:
# torch 사용가능한지 확인

import torch
x = torch.rand(5, 3)
print(x)

print(torch.__version__)
print(torch.cuda.is_available())

tensor([[0.6697, 0.2794, 0.4329],
        [0.4944, 0.3505, 0.8900],
        [0.0087, 0.2734, 0.0721],
        [0.5704, 0.3738, 0.0837],
        [0.1363, 0.8818, 0.6198]])
1.10.2+cu102
True


In [ ]:
# import matplotlib.pyplot as plt
# import gluonnlp as nlp

In [6]:
df = pd.read_csv('naver_shopping_all_220427.csv')

In [7]:
y = [0 if rate < 3 else 1 for rate in df.label] # 3점 미만이면 0(부정), 3점 초과이면 1(긍정)으로 라벨 생성

In [70]:
#평가 데이터셋 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df['text'].tolist(), y, random_state=0)

#학습, 검증 데이터셋분리
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

print(len(X_train), len(X_val), len(X_test))
print(len(y_train), len(y_val), len(y_test))

112500 37500 50000
112500 37500 50000


### bert-base-multilingual-cased를 이용한 미세조정학습


In [71]:
# 실습 전 GPU 정리
del model
del trainer
torch.cuda.empty_cache()

In [72]:
import torch
print(torch.cuda.is_available())
device = torch.device('cuda')
device = torch.device("cuda:0")
print(torch.__version__)
#model.to(device)
#

True
1.10.2+cu102


In [73]:
#정확도 측정을 위한 준비 단계 (... 잘 모르겠음..)

import torch
from datasets import load_metric
metric = load_metric("accuracy")


# 정확도 계산 함수 생성

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [74]:
# 토큰화 준비&확인
from transformers import pipeline
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize('안녕하세요. 반갑습니다.'))

inputs = tokenizer('안녕하세요. 반갑습니다.')
print(inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [75]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [76]:
# 토큰화
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

In [77]:
# dataset 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

In [78]:
# bert-base-multilingual-cased 사전학습 모형으로부터 분류기 모형을 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased").to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [79]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    evaluation_strategy='steps',
    eval_steps = 500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    save_total_limit=1    #checkpoint 갯수
)

In [47]:
# checkpoint_callback = ModelCheckpoint(
#     filepath='./checkpoint_test',
#     save_top_k=3,
#     monitor='val_acc',
#     mode='max'
# )

In [80]:
# Trainer 객체생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    )

In [81]:
trainer.train()

/tmp/ipykernel_11135/2497499804.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
500,0.553600,0.481021,0.813360,203.460700,184.311000
1000,0.529100,0.737283,0.503227,206.998000,181.161000
1500,0.539900,0.479215,0.811067,204.227700,183.619000
2000,0.636900,0.667264,0.615013,215.121200,174.320000
2500,0.572500,0.622497,0.747707,206.774200,181.357000
3000,0.518200,0.661193,0.569147,207.713400,180.537000
3500,0.702500,0.695135,0.520507,203.757100,184.043000
4000,0.691500,0.685652,0.496773,203.796400,184.007000
4500,0.695200,0.689115,0.510347,209.343200,179.132000
5000,0.565600,0.489309,0.815520,210.486200,178.159000


Attempted to log scalar metric loss:
0.5536
Attempted to log scalar metric learning_rate:
4.856764305664972e-05
Attempted to log scalar metric epoch:
0.04
Attempted to log scalar metric eval_loss:
0.4810209274291992
Attempted to log scalar metric eval_accuracy:
0.81336
Attempted to log scalar metric eval_runtime:
203.4607
Attempted to log scalar metric eval_samples_per_second:
184.311
Attempted to log scalar metric epoch:
0.04


/tmp/ipykernel_11135/2497499804.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Attempted to log scalar metric loss:
0.5291
Attempted to log scalar metric learning_rate:
4.6777196877461866e-05
Attempted to log scalar metric epoch:
0.07
Attempted to log scalar metric eval_loss:
0.7372829914093018
Attempted to log scalar metric eval_accuracy:
0.5032266666666667
Attempted to log scalar metric eval_runtime:
206.998
Attempted to log scalar metric eval_samples_per_second:
181.161
Attempted to log scalar metric epoch:
0.07


...

/tmp/ipykernel_11135/2497499804.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Attempted to log scalar metric loss:
0.5834
Attempted to log scalar metric learning_rate:
2.2559621857766956e-07
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
0.5855880975723267
Attempted to log scalar metric eval_accuracy:
0.6892
Attempted to log scalar metric eval_runtime:
209.9945
Attempted to log scalar metric eval_samples_per_second:
178.576
Attempted to log scalar metric epoch:
1.0


/tmp/ipykernel_11135/2497499804.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Attempted to log scalar metric train_runtime:
14813.9026
Attempted to log scalar metric train_samples_per_second:
0.949
Attempted to log scalar metric total_flos:
1.500651376875e+16
Attempted to log scalar metric epoch:
1.0


TrainOutput(global_step=14063, training_loss=0.5994022111418678, metrics={'train_runtime': 14813.9026, 'train_samples_per_second': 0.949, 'total_flos': 1.500651376875e+16, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': -1048576, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 1048576, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 78082048, 'train_mem_gpu_alloc_delta': 2166102528, 'train_mem_cpu_peaked_delta': 745082880, 'train_mem_gpu_peaked_delta': 877608448})

In [82]:
trainer.save_model('questionDetection_model_v0.0.1')

In [83]:
trainer.evaluate(eval_dataset=test_dataset)

/tmp/ipykernel_11135/2497499804.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Attempted to log scalar metric eval_loss:
0.5845708250999451
Attempted to log scalar metric eval_accuracy:
0.69154
Attempted to log scalar metric eval_runtime:
275.5246
Attempted to log scalar metric eval_samples_per_second:
181.472
Attempted to log scalar metric epoch:
1.0


{'eval_loss': 0.5845708250999451,
 'eval_accuracy': 0.69154,
 'eval_runtime': 275.5246,
 'eval_samples_per_second': 181.472,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 8192,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_peaked_delta': 62691328}